<a href="https://colab.research.google.com/github/markwang0/GeoFlood/blob/WIP-pygeoflood-class/examples/geoflood_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### Install in-development branch of GeoFlood


In [1]:
%%capture

inColab = 'google.colab' in str(get_ipython())
if inColab:
    %pip install git+https://github.com/markwang0/geoflood@WIP-pygeoflood-class#egg=pygeoflood
else:
    %pip install -e ..
    pass


### Run GeoFlood operations


In [2]:
from pathlib import Path
from pygeoflood import PyGeoFlood

data_dir = Path("data")
if not data_dir.is_dir():
    data_dir.mkdir()
my_dem_path = Path(data_dir, "OC1mTest.tif")
# download DEM if it doesn't exist
if not my_dem_path.is_file():
    !curl https://utexas.box.com/shared/static/id0in7w8yn38o59sexa50aqunsr7mzk0.tif -Lso data/OC1mTest.tif

geoflood_onion = PyGeoFlood(dem_path=my_dem_path)
print(geoflood_onion)


PyGeoFlood(
    dem_path="data/OC1mTest.tif",
    project_dir="data",
)


In [3]:
# apply nonlinear filter to DEM
geoflood_onion.apply_nonlinear_filter()

Computing slope of raw DTM
DEM slope array shape: (3378, 4750)
Computing lambda = q-q-based nonlinear filtering threshold
DEM smoothing Quantile: 0.9
Edge Threshold Value: 0.34356024861335754
Filtered DEM written to data/OC1mTest_filtered.tif

apply_nonlinear_filter completed in 9.8020 seconds


In [4]:
# calculate slope
geoflood_onion.calculate_slope()

 slope statistics
 angle min: 0.0
 angle max: 54.863846043721324
 mean slope: 0.13960592
 stdev slope: 0.15432632
Slope raster written to data/OC1mTest_slope.tif

calculate_slope completed in 1.2548 seconds


In [5]:
# calculate curvature
geoflood_onion.calculate_curvature()

 using geometric curvature
 curvature statistics
 non-nan curvature cell number: 16045500
 non-nan finite curvature cell number:  mean:  -0.00031314598
 standard deviation:  0.112147875
Curvature raster written to data/OC1mTest_curvature.tif

calculate_curvature completed in 1.2137 seconds


In [6]:
# fill depressions on filtered DEM
geoflood_onion.fill_depressions()

Filled DEM written to data/OC1mTest_filled.tif

fill_depressions completed in 7.7034 seconds


In [7]:
# calculate MFD flow accumulation
geoflood_onion.calculate_mfd_flow_accumulation()

MFD flow accumulation raster written to data/OC1mTest_mfd_fac.tif

calculate_mfd_flow_accumulation completed in 4.4118 seconds


In [8]:
# calculate D8 flow directions
geoflood_onion.calculate_d8_flow_direction()

D8 flow direction raster written to data/OC1mTest_d8_fdr.tif

calculate_d8_flow_direction completed in 1.6816 seconds


In [9]:
# calculate outlets
geoflood_onion.find_outlets()

Outlets raster written to data/OC1mTest_outlets.tif

find_outlets completed in 0.2143 seconds


In [10]:
# delineate basins
geoflood_onion.delineate_basins()

Basins raster written to data/OC1mTest_basins.tif

delineate_basins completed in 0.9339 seconds


In [11]:
# define skeletons
geoflood_onion.define_skeleton(
    write_curvature_skeleton=True,
    write_flow_skeleton=True,
)

Curvature mean:  -0.00031413726
Curvature standard deviation:  0.11232527
Curvature Projection: EPSG:6343
Mean upstream flow:  2617.3486
Flow skeleton written to data/OC1mTest_flow_skeleton.tif
Curvature skeleton written to data/OC1mTest_curvature_skeleton.tif
Combined skeleton written to data/OC1mTest_combined_skeleton.tif

define_skeleton completed in 1.0844 seconds


In [12]:
# geodesic distance (not needed for GeoFlood workflow)
geoflood_onion.calculate_geodesic_distance(write_cost_function=True)

Performing fast marching
FMM iteration 1/32: RAM usage: 11.13/16.00 GB (69.6%)
FMM iteration 2/32: RAM usage: 11.04/16.00 GB (69.0%)
FMM iteration 3/32: RAM usage: 10.99/16.00 GB (68.7%)
FMM iteration 4/32: RAM usage: 10.99/16.00 GB (68.7%)
FMM iteration 5/32: RAM usage: 11.00/16.00 GB (68.8%)
FMM iteration 6/32: RAM usage: 10.99/16.00 GB (68.7%)
FMM iteration 7/32: RAM usage: 11.04/16.00 GB (69.0%)
FMM iteration 8/32: RAM usage: 11.04/16.00 GB (69.0%)
FMM iteration 9/32: RAM usage: 11.04/16.00 GB (69.0%)
FMM iteration 10/32: RAM usage: 11.04/16.00 GB (69.0%)
FMM iteration 11/32: RAM usage: 11.04/16.00 GB (69.0%)
FMM iteration 12/32: RAM usage: 11.04/16.00 GB (69.0%)
FMM iteration 13/32: RAM usage: 10.99/16.00 GB (68.7%)
FMM iteration 14/32: RAM usage: 11.05/16.00 GB (69.1%)
FMM iteration 15/32: RAM usage: 11.05/16.00 GB (69.1%)
FMM iteration 16/32: RAM usage: 11.05/16.00 GB (69.1%)
FMM iteration 17/32: RAM usage: 11.05/16.00 GB (69.1%)
FMM iteration 18/32: RAM usage: 11.05/16.00 GB (6

In [13]:
# get channel heads
geoflood_onion.identify_channel_heads()

Locating skeleton end points
Counting the number of elements of each connected component
Skeleton region size threshold: 258.7368421052632
Continuing to locate skeleton endpoints
Channel heads shapefile written to data/OC1mTest_channel_heads.shp

identify_channel_heads completed in 2.3973 seconds


In [14]:
# set path to NFIE (NHD MR) flowline
# full dataset here: https://www.arcgis.com/home/webmap/viewer.html?webmap=9766a82973b34f18b43dafa20c5ef535
# download demo flowline if it doesn't exist
geoflood_onion.flowline_path = Path(data_dir, "Flowline.shp")
if not geoflood_onion.flowline_path.is_file():
    !curl -sL https://utexas.box.com/shared/static/s1vlnymonbidnhul52oja7s83ky1212j.tgz | tar -xz -C $data_dir
# get flowline endpoints
geoflood_onion.find_endpoints()

Endpoints csv written to data/OC1mTest_endpoints.csv

find_endpoints completed in 0.0264 seconds


In [15]:
# get binary HAND array
geoflood_onion.calculate_binary_hand()

Binary HAND raster written to data/OC1mTest_binary_hand.tif

calculate_binary_hand completed in 3.2747 seconds


In [16]:
# get custom flowline from NHD HR dataset (optional)
geoflood_onion.custom_flowline_path = Path(
    data_dir, "NHDPlus_H_1209_HU4_GDB.gdb"
)
geoflood_onion.rasterize_custom_flowline(layer="NHDFlowline")

Custom flowline raster written to data/OC1mTest_custom_flowline.tif

rasterize_custom_flowline completed in 0.8900 seconds


In [17]:
# extract channel network
geoflood_onion.extract_channel_network()

Cost function weights:
curvature          0.2520 (mean flow accumulation)
mfd_fac            1.0000
binary_hand        0.7500
custom_flowline    0.0000

Cost min: 0.5028838188775905
Cost max: inf
cost shape: (3378, 4750)
Creating path 1/1: RAM usage: 11.22/16.00 GB (70.1%)
Number of endpoints: 1
Channel network raster written to data/OC1mTest_channel_network.tif
Channel network vector written to data/OC1mTest_channel_network.shp

extract_channel_network completed in 1.9264 seconds


In [18]:
# calculate HAND
geoflood_onion.calculate_hand()

HAND raster written to data/OC1mTest_HAND.tif

calculate_hand completed in 4.8302 seconds


In [19]:
# # segment flowline
# geoflood_onion.segment_flowline()

In [20]:
geoflood_onion

PyGeoFlood(
    dem_path="data/OC1mTest.tif",
    project_dir="data",
    filtered_dem_path="data/OC1mTest_filtered.tif",
    slope_path="data/OC1mTest_slope.tif",
    curvature_path="data/OC1mTest_curvature.tif",
    filled_path="data/OC1mTest_filled.tif",
    mfd_fac_path="data/OC1mTest_mfd_fac.tif",
    d8_fdr_path="data/OC1mTest_d8_fdr.tif",
    basins_path="data/OC1mTest_basins.tif",
    outlets_path="data/OC1mTest_outlets.tif",
    flow_skeleton_path="data/OC1mTest_flow_skeleton.tif",
    curvature_skeleton_path="data/OC1mTest_curvature_skeleton.tif",
    combined_skeleton_path="data/OC1mTest_combined_skeleton.tif",
    cost_function_geodesic_path="data/OC1mTest_cost_function_geodesic.tif",
    geodesic_distance_path="data/OC1mTest_geodesic_distance.tif",
    channel_heads_path="data/OC1mTest_channel_heads.shp",
    flowline_path="data/Flowline.shp",
    endpoints_path="data/OC1mTest_endpoints.csv",
    binary_hand_path="data/OC1mTest_binary_hand.tif",
    custom_flowline_path="d

In [21]:
# # set path to NFIE (NHD MR) flowline
# # full dataset here: https://www.arcgis.com/home/webmap/viewer.html?webmap=9766a82973b34f18b43dafa20c5ef535
# # download demo flowline if it doesn't exist
# geoflood_onion.catchment_path = Path(data_dir, "Catchment.shp")
# if not geoflood_onion.catchment_path.is_file():
#     !curl -sL https://utexas.box.com/shared/static/jzawwvmb2tjl4yqtnitgvfjcc4ho4tg5.tgz | tar -xz -C $data_dir
# # calculate stream segment attributes (ID, slope, length, area)
# geoflood_onion.river_attributes()